In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
import delfi.distribution as dd
import numpy as np
import pickle
import time
import scipy.stats as st
import os 
from lfimodels.balancednetwork.BalancedNetworkSimulator import BalancedNetwork
from lfimodels.balancednetwork.BalancedNetworkStats import BalancedNetworkStats, Identity

mpl_params = {'legend.fontsize': 14,
                      'axes.titlesize': 20,
                      'axes.labelsize': 17,
                      'xtick.labelsize': 12,
                      'ytick.labelsize': 12,
             'figure.figsize' : (15, 5)}

mpl.rcParams.update(mpl_params)

In [ ]:
save_figure = True

inference_method = 'snpe'
fileformat = '.png'
dpi = 300
param_label = r'$w_{ei}$'
inference_param_name = 'wei'

# set name to find the folder 
simulation_name = '15140527888578181_snpe_cJie_r10_n1000_rcl5'
path_to_save_folder = os.path.join('../results', simulation_name)

In [ ]:
# load data 
time_str = simulation_name[:simulation_name.find('_')]
fullname = os.path.join(path_to_save_folder, simulation_name + '.p')


# load data 
with open(fullname, 'rb') as handle:
    result_dict = pickle.load(handle)

print(result_dict.keys())

try: 
    true_params, stats_obs, nrounds, ntrain, posterior, out, trn_data, prior, posteriors = result_dict.values()
    theta = np.linspace(prior.lower[0], prior.upper[0], 1000)
except ValueError:
    try: 
        true_params, stats_obs, nrounds, ntrain, posterior, out, trn_data, prior = result_dict.values()            
        theta = np.linspace(prior.lower[0], prior.upper[0], 1000)
    except ValueError: 
        true_params, stats_obs, nrounds, ntrain, posterior, out, trn_data= result_dict.values()            
        theta = np.linspace(0, 5, 1000)

## Plot single figure with posterior over rounds 

In [ ]:
for idx, posterior in enumerate(posteriors): 
    n_components = len(posterior.a)
    means = [posterior.xs[c].m for c in range(n_components)]
    Ss = [posterior.xs[c].S for c in range(n_components)]

    sub_means = [[means[c][0]] for c in range(n_components)]
    sub_cov = np.asarray([Ss[c] for c in range(n_components)])
    pdf = dd.mixture.MoG(a=posterior.a, ms=sub_means, Ss=sub_cov)
    post_pdf = pdf.eval(theta[:, np.newaxis], log=False)
    
    if idx == nrounds-1: 
        plt.plot(theta, post_pdf, label='round {}, cl'.format(idx + 1))
    elif idx > 4: 
        plt.plot(theta, post_pdf, label='round {}, cl'.format(idx + 1), alpha=.6, linestyle='-')
    else:
        plt.plot(theta, post_pdf, label='round {}'.format(idx + 1), alpha=.6, linestyle='--')
    plt.xlabel(param_label)
    
plt.axvline(x=true_params[0], label=r'observed {}'.format(param_label), linestyle='-', color='k', alpha=.5)
plt.legend()
plt.title('Posteriors over rounds');

plt.tight_layout()
filename = time_str + '_{}_posteriors'.format(inference_param_name)
if save_figure and os.path.exists(path_to_save_folder): 
    destination = os.path.join(path_to_save_folder, filename + fileformat)
    plt.savefig(destination, dpi=dpi)
    print('saved file in {}'.format(destination));

## Plotting the summary stats 

In [ ]:
plt.figure(figsize=(15, 10))
p = trn_data[0][0].flatten()
ss = trn_data[0][1]
titles = ['Fano factor', 'mean rate', 'kurtosis', 'positive pairwise corr', 
          '0 lag auto/cross corr', '10 lag auto/cross corr', '20 lag auto/cross corr']
ylabels = ['ff', 'rate', 'kurt', 'corr prop',
                                    'corr0',
                                    'corr10',
                                    'corr20']

labels = ['ffE', 'ffI', 'rateE', 'rateI', 'kurtE', 'kurtI', 'corrE', 'corrI',
                                    'EE', 'EI', 'II',
                                    'EE', 'EI', 'IE', 'II',
                                    'EE', 'EI', 'IE', 'II']
plot_idx = [1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7]

for idx, s in enumerate(ss.T): 
    plt.subplot(3, 3, plot_idx[idx])
    plt.plot(p, s, 'o', label=labels[idx], alpha=.6)
    plt.title(titles[plot_idx[idx] - 1])
    plt.ylabel(ylabels[plot_idx[idx] - 1])
    if idx > 7: 
        plt.xlabel(param_label)
    plt.legend()
    plt.axvline(x=true_params[0])
plt.tight_layout()

filename = time_str + '_{}_summary_stats'.format(inference_param_name)
if save_figure and os.path.exists(path_to_save_folder): 
    destination = os.path.join(path_to_save_folder, filename + fileformat)
    plt.savefig(destination, dpi=200)
    print('saved file in {}'.format(destination));

In [ ]:
posterior.mean

## Posterior predictive checking 

Generate samples from the posterior and simulate them. The resulting data should be near the observed data. 

In [ ]:
# make sure to change this accordingly!!! 
ree = 2.5

m = BalancedNetwork(inference_params=[inference_param_name], n_servers=3, duration=3., first_port=8010, 
                    save_raster_plots=False,
                    calculate_stats=True, ree=ree,
                    save_folder='/Users/Jan/Dropbox/Master/mackelab/code/lfi-experiments/balancednetwork/figures/simulation_raster_plots/')
s = Identity()

In [ ]:
# calculate mean of mixture 
mean = np.sum([a * m[0] for a, m in zip(posterior.a, means)])
# calculate variance of mixture 
sum_squared = np.sum([a * m[0]**2 for a, m in zip(posterior.a, means)])
squared_sum = mean**2
std = np.sum([a * Ss[0][0] for a, m in zip(posterior.a, Ss)]) + sum_squared - squared_sum

In [ ]:
m.start_server()
# generate theta +-3, 2, 1 0 stds away from mode
param_list = [[mean + i * std] for i in [-2, 0, 2]]
sum_stats = []
# simulate and collect sum stats
data = m.gen(param_list)

In [ ]:
sum_stats = s.calc_all(data)

## Compare only simulation of mean posterior vs. original parameters

In [ ]:
labels = ['ffE', 'ffI', 'rateE', 'rateI', 'kurtE', 'kurtI', 'corrE', 'corrI',
                                    '0EE', '0EI', '0II',
                                    '10EE', '10EI', '10IE', '10II',
                                    '20EE', '20EI', '20IE', '20II']

ax = plt.plot(stats_obs.T, '-*', lw=3., label='observed')

plt.plot(sum_stats[0].flatten(), '--o', label='- 2 std posterior', alpha=.6)
plt.plot(sum_stats[1].flatten(), '--o', label='mean posterior')
plt.plot(sum_stats[2].flatten(), '--o', label='+ 2 std posterior', alpha=.6)

plt.xticks(np.arange(19), labels)
plt.gca().tick_params(axis='x')
plt.legend()

plt.ylabel('summary stats [a.u.]')
plt.title('Comparison of observed and posterior summary statistics')

filename = time_str + '_{}_stats_comparison'.format(inference_param_name)

plt.tight_layout()
if save_figure and os.path.exists(path_to_save_folder): 
    destination = os.path.join(path_to_save_folder, filename + fileformat)
    plt.savefig(destination, dpi=dpi)
    print('saved file in {}'.format(destination));